In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import pandas as pd

In [12]:
import os
api_key = os.environ.get("OPENAI_API_KEY")

In [13]:
players_file = pd.read_csv('data/merged.csv')

In [14]:
print((players_file))

          player_name player_team_id player_position_id  rank pos_rank  salary
0    patrickmahomesii             KC                 QB     1      QB1  9000.0
1          jalenhurts            PHI                 QB     2      QB2  7800.0
2           joshallen            BUF                 QB     3      QB3  7900.0
3        lamarjackson            BAL                 QB     4      QB4  8000.0
4           joeburrow            CIN                 QB     5      QB5  7100.0
..                ...            ...                ...   ...      ...     ...
707           falcons            ATL                DST    28    DST28  3200.0
708         cardinals            ARI                DST    29    DST29  2500.0
709            texans            HOU                DST    30    DST30  2100.0
710             bears            CHI                DST    31    DST31  2900.0
711           raiders             LV                DST    32    DST32  2300.0

[712 rows x 6 columns]


In [15]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo-16k", temperature=.2)

In [16]:
TEAM_PROMPT = PromptTemplate(
input_variables=["context"],
template = """You are a dedicated Sports Analytics Assistant. Your primary goal is to identify and assemble the most optimal sports team lineup, ensuring it stays below a Salary Cap of $50,000, using advanced analytical techniques. Omit players labeled "No Salary". If any pertinent data isn't in the context, kindly notify that the information is unavailable. Strict compliance with guidelines is imperative.

**Salary Cap:** $50,000

**Data Attributes Within {context}:**

rank: Expert Consensus Ranking (the lower, the more favorable).
pos_rank: Player's ranking by their specific position (lower numbers indicate better performance).
salary: Player's financial remuneration.
Procedure:

**Task**

1. Analyze the player statistics in {context} and determine the most statistically advantageous lineup comprising::
* One QB
* Two RBs
* Three WRs
* One TE
* One FLEX (can be a RB, WR, or TE)
* One DST
2. Calculate cumulative salaries to verify it remains beneath the set cap.
3. If the assembled team adheres to the budget, present this lineup.
4. If the budget is surpassed, make informed adjustments to the team.
5. Cycle through steps 2-4, employing analytical methodologies until you curate a lineup that is both optimal and under budget.

**Provide the following as an outcome:**
List of selected players categorized by their positions, ensuring they are the best picks from {context} while staying under the salary cap.

""")

In [17]:
PLAYER_PROMPT = PromptTemplate(
input_variables=["context"],
template = """As a Sports Analytics expert, your role revolves around answering questions that pertain to a dataset containing information about various sports players. Your analysis should focus on players with a valid salary (excluding those with "No Salary" listed). The dataset, which you will find in the variable `{context}`, includes the following pertinent fields:

1. `rank_ecr`: This denotes the Expert Consensus ranking.
2. `pos_rank`: This field signifies a player's ranking within their specific position.
3. `salary`: The salary cost associated with each player.


Here's what you need to do:

1. Filter the dataset to include only WRs (players with player_position_id = 'WR') and exclude players with "No Salary" listed.
2. Calculate a "value score" for each WR using the formula: (average rank for all WRs / player's rank) * player's salary.
3. Identify and return the top 5 WRs with the lowest value scores.
4. Please provide the list of these 5 undervalued WRs, including their names, rankings, and salaries.

Now, let's apply this process to the dataset stored in {context} and return the desired information.
""")

In [18]:
# question = "Who are the 5 most undervalued WRs?"

In [19]:
chain = LLMChain(llm=llm, prompt=TEAM_PROMPT)

In [20]:
print(chain.run(context=players_file))


To assemble the most optimal sports team lineup while staying under the salary cap, we will follow the steps outlined in the task. Here is the solution:

1. Analyze the player statistics and determine the most statistically advantageous lineup comprising:
   - One QB
   - Two RBs
   - Three WRs
   - One TE
   - One FLEX (can be a RB, WR, or TE)
   - One DST

To do this, we will select the players with the lowest rank (Expert Consensus Ranking) and pos_rank (Player's ranking by their specific position) for each position, while omitting players labeled "No Salary".

QB:
- Select the player with the lowest rank and pos_rank among QBs: Jalen Hurts (PHI) - Salary: $7800

RB:
- Select the two players with the lowest rank and pos_rank among RBs: Najee Harris (PIT) - Salary: $6000 and Antonio Gibson (WAS) - Salary: $5800

WR:
- Select the three players with the lowest rank and pos_rank among WRs: Justin Jefferson (MIN) - Salary: $7000, Tee Higgins (CIN) - Salary: $5500, and Jerry Jeudy (DEN) -